# Settings



In [1]:
'''To display all output results of a Jupyter cell.'''
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

'To display all output results of a Jupyter cell.'

In [2]:
'''To ensure that the output results are not truncated.'''
#pd.options.display.max_rows = 4000

'To ensure that the output results are not truncated.'

In [49]:
%autosave 180

Autosaving every 180 seconds


# Import of packages

In [3]:
'''To install the required packages that are not included in Colaboratory.'''
#Installation of packages that are not included in Colaboratory
!pip install kneed
!pip install osmnx

  Using cached matplotlib-3.3.4-cp38-cp38-macosx_10_9_x86_64.whl (8.5 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3


In [4]:
'''To install a specific version of the matplotlib package.'''
!pip install matplotlib==3.1.3

  Using cached matplotlib-3.1.3-cp38-cp38-macosx_10_9_x86_64.whl (13.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.4
    Uninstalling matplotlib-3.3.4:
      Successfully uninstalled matplotlib-3.3.4
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

osmnx 1.0.1 requires matplotlib>=3.3, but you'll have matplotlib 3.1.3 which is incompatible.


In [5]:
'''To import the required packages.'''
import pandas as pd
import numpy as np
import networkx as nx
import collections
import matplotlib.pyplot as plt
import math

from kneed import KneeLocator
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

import osmnx as ox

# Import of the Belgian railway datasets

In [6]:
'''To register the GitHub link with the Belgian data as a variable.'''
datalink = "https://raw.githubusercontent.com/polkuleuven/thesis/main/"

In [7]:
'''To import the agency dataset that contains limited information about Belgian NMBS/SNCB railway agency.'''
agency = pd.read_csv(datalink + "agency.csv", sep=",")
#agency

In [8]:
'''To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the Belgian railway stations.'''
stops = pd.read_csv(datalink + "stops.csv", encoding='iso-8859-1', sep=",")
stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code
0,8015345,NaN,Aachen Hbf (DE),NaN,50.77083,6.105277,NaN,NaN,0,NaN,NaN
1,8200100,NaN,Luxembourg (LU),NaN,49.60000,6.133333,NaN,NaN,0,NaN,NaN
2,8200101,NaN,Dommeldange (LU),NaN,49.63390,6.136765,NaN,NaN,0,NaN,NaN
3,8200102,NaN,Pfaffenthal-Kirchberg (LU),NaN,49.61913,6.132853,NaN,NaN,0,NaN,NaN
4,8200110,NaN,Mersch (LU),NaN,49.74889,6.106111,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2635,8896909,NaN,Izegem,NaN,50.92115,3.212089,NaN,NaN,0,S8896909,NaN
2636,S8896925,NaN,Ingelmunster,NaN,50.91433,3.255416,NaN,NaN,1,NaN,NaN
2637,8896925_2,NaN,Ingelmunster,NaN,50.91433,3.255416,NaN,NaN,0,S8896925,2
2638,8896925_1,NaN,Ingelmunster,NaN,50.91433,3.255416,NaN,NaN,0,S8896925,1


In [9]:
'''To import the translations dataset that provides the French-, Dutch-, German- and English-language translations of the Belgian railway stations.'''
translations = pd.read_csv(datalink + "translations.csv", encoding='iso-8859-1', sep=",")
#translations

In [10]:
'''To import the transfers dataset that gives the minimum transfer time to switch routes at each Belgian railway station.'''
transfers = pd.read_csv(datalink + "transfers.csv", sep=",")
transfers

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_trip_id,to_trip_id
0,8015345,8015345,2,300,NaN,NaN
1,8200100,8200100,2,300,NaN,NaN
2,8200101,8200101,2,300,NaN,NaN
3,8200102,8200102,2,300,NaN,NaN
4,8200110,8200110,2,300,NaN,NaN
...,...,...,...,...,...,...
606,8896503,8896503,2,300,NaN,NaN
607,8896735,8896735,2,300,NaN,NaN
608,8896800,8896800,2,300,NaN,NaN
609,8896909,8896909,2,300,NaN,NaN


In [11]:
'''To import the routes dataset that provides the id, the name and the type of vehicle used for all Belgian railway routes.'''
routes = pd.read_csv(datalink + "routes.csv", encoding='iso-8859-1', sep=",")
routes

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,1,NMBS/SNCB,IC,Liège-Guillemins -- Liers,NaN,103,NaN,NaN,NaN
1,10,NMBS/SNCB,BUS,La Louvière-Sud -- Manage,NaN,700,NaN,NaN,NaN
2,100,NMBS/SNCB,IC,Enghien -- Tournai,NaN,103,NaN,NaN,NaN
3,101,NMBS/SNCB,BUS,Anvers-Central -- Tongres,NaN,700,NaN,NaN,NaN
4,102,NMBS/SNCB,BUS,Flémalle-Haute -- Huy,NaN,700,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
674,95,NMBS/SNCB,BUS,Zottegem -- Grammont,NaN,700,NaN,NaN,NaN
675,96,NMBS/SNCB,BUS,Hasselt -- Tongres,NaN,700,NaN,NaN,NaN
676,97,NMBS/SNCB,IC,Hal -- Brussels Airport-Zaventem,NaN,103,NaN,NaN,NaN
677,98,NMBS/SNCB,BUS,Charleroi-Sud -- Tamines,NaN,700,NaN,NaN,NaN


In [12]:
'''To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the Belgian railway route.
The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'''
trips = pd.read_csv(datalink + "trips.csv", encoding='iso-8859-1', sep=",")
trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,trip_type
0,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,106,NaN,1,NaN,1
1,2,1,88____:046::8841004:8863008:20:722:20201212,Namur,4977,NaN,1,NaN,1
2,3,2,88____:007::8841673:8845005:18:723:20200520,Luxembourg (LU),106,NaN,2,NaN,1
3,3,2,88____:007::8845005:8200136:3:734:20200520,Luxembourg (LU),106,NaN,2,NaN,1
4,3,21,88____:007::8845005:8200136:3:734:20200430,Luxembourg (LU),106,NaN,3,NaN,1
...,...,...,...,...,...,...,...,...,...
25953,676,345,88____:046::8865003:8864345:6:1730:20201212,Marloie,987,NaN,20348,NaN,1
25954,676,340,88____:046::8865003:8864345:6:1832:20201211,Marloie,988,NaN,20349,NaN,1
25955,676,345,88____:046::8865003:8864345:6:1930:20201212,Marloie,989,NaN,20350,NaN,1
25956,676,340,88____:046::8865003:8864345:6:2032:20201211,Marloie,990,NaN,20351,NaN,1


In [13]:
'''To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
In addition, for all the trips the arrival and departure times at the stations served are given.'''
stop_times = pd.read_csv(datalink + "stop_times-1.csv", sep=",")
stop_times = stop_times.append(pd.read_csv(datalink + "stop_times-2.csv", sep=","), ignore_index=True)
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,88____:007::8841673:8841004:6:559:20201212,05:35:00,05:35:00,8841673,1,NaN,0,1,NaN
1,88____:007::8841673:8841004:6:559:20201212,05:38:00,05:39:00,8841665,2,NaN,0,0,NaN
2,88____:007::8841673:8841004:6:559:20201212,05:44:00,05:45:00,8841608,3,NaN,0,0,NaN
3,88____:007::8841673:8841004:6:559:20201212,05:50:00,05:51:00,8841525,4,NaN,0,0,NaN
4,88____:007::8841673:8841004:6:559:20201212,05:54:00,05:55:00,8841558,5,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
344417,88____:046::8865003:8864345:6:2130:20201212,21:06:00,21:06:00,8865227,2,NaN,0,0,NaN
344418,88____:046::8865003:8864345:6:2130:20201212,21:16:00,21:16:00,8864337,3,NaN,0,0,NaN
344419,88____:046::8865003:8864345:6:2130:20201212,21:21:00,21:21:00,8864311,4,NaN,0,0,NaN
344420,88____:046::8865003:8864345:6:2130:20201212,21:24:00,21:25:00,8864006,5,NaN,0,0,NaN


In [14]:
'''To import the calendar dataset that gives the first and last date of all data observations.'''
calendar = pd.read_csv(datalink + "calendar.csv", sep=",")
#calendar

In [15]:
'''To import the calendar_dates dataset that gives for each service_id all the exact dates when that service_id is valid.'''
calendar_dates = pd.read_csv(datalink + "calendar_dates.csv", sep=",")
#calendar_dates

In [16]:
''' ??? '''
stop_time_overrides = pd.read_csv(datalink + "stop_time_overrides.csv", sep=",")
stop_time_overrides

,trip_id,stop_sequence,service_id,stop_id
0,88____:007::8841673:8841004:6:559:20201212,1,3218,8841673_3
1,88____:007::8841673:8841004:6:559:20201212,2,2645,8841665_2
2,88____:007::8841673:8841004:6:559:20201212,3,2645,8841608_2
3,88____:007::8841673:8841004:6:559:20201212,4,2645,8841525_1
4,88____:007::8841673:8841004:6:559:20201212,5,2645,8841558_1
...,...,...,...,...
171160,88____:046::8865003:8864345:6:2130:20201212,2,345,8865227_2
171161,88____:046::8865003:8864345:6:2130:20201212,3,345,8864337_1
171162,88____:046::8865003:8864345:6:2130:20201212,4,345,8864311_2
171163,88____:046::8865003:8864345:6:2130:20201212,5,345,8864006_6


# Cleaning of the Belgian railway data

In [17]:
'''To eliminate the stop_ids in the stops dataset that contain an underscore or that start with a character 'S'. '''
stops_cleaned = stops[(~stops['stop_id'].str.contains('_')) & (~stops['stop_id'].str.contains('S'))]

'''To modify the object datatype of the stop_id column to the numpy int64 datatype'''
stops_cleaned.loc[:,'stop_id'] = stops_cleaned.loc[:,'stop_id'].astype(np.int64)
stops_cleaned

/Users/pol/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code
0,8015345,NaN,Aachen Hbf (DE),NaN,50.77083,6.105277,NaN,NaN,0,NaN,NaN
1,8200100,NaN,Luxembourg (LU),NaN,49.60000,6.133333,NaN,NaN,0,NaN,NaN
2,8200101,NaN,Dommeldange (LU),NaN,49.63390,6.136765,NaN,NaN,0,NaN,NaN
3,8200102,NaN,Pfaffenthal-Kirchberg (LU),NaN,49.61913,6.132853,NaN,NaN,0,NaN,NaN
4,8200110,NaN,Mersch (LU),NaN,49.74889,6.106111,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2621,8896503,NaN,Ypres,NaN,50.84740,2.876590,NaN,NaN,0,S8896503,NaN
2626,8896735,NaN,Poperinge,NaN,50.85445,2.736345,NaN,NaN,0,S8896735,NaN
2631,8896800,NaN,Roulers,NaN,50.94903,3.130415,NaN,NaN,0,S8896800,NaN
2635,8896909,NaN,Izegem,NaN,50.92115,3.212089,NaN,NaN,0,S8896909,NaN


In [18]:
'''To merge a selection of the trips dataset and a selection of the routes dataset on route_id'''
trip_route_short_name = pd.merge(trips[['route_id','service_id','trip_id', 'trip_headsign']], routes[['route_id', 'route_short_name', 'route_long_name']], on='route_id')

'''To select the trips that belong to the routes that have a route_short_name that begins with an 'S' or is equal to 'IC', 'L' or 'P.'''
allowed_route_type = {'IC', 'L', 'P'}
filtered_trips = trip_route_short_name[(trip_route_short_name['route_short_name'].isin(allowed_route_type)) | (trip_route_short_name['route_short_name'].str.startswith('S'))]
filtered_trips = filtered_trips.drop(columns=['route_short_name'])
filtered_trips

,route_id,service_id,trip_id,trip_headsign,route_long_name
0,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers
1,1,48,88____:007::8841673:8841004:6:528:20200524,Liège-Guillemins,Liège-Guillemins -- Liers
2,1,48,88____:007::8841004:8841673:6:857:20200524,Liers,Liège-Guillemins -- Liers
3,1,48,88____:007::8841673:8841004:6:728:20200524,Liège-Guillemins,Liège-Guillemins -- Liers
4,1,48,88____:007::8841004:8841673:6:1057:20200524,Liers,Liège-Guillemins -- Liers
...,...,...,...,...,...
25953,679,716,84____:007::8400131:8400280:3:1720:20200705,Den Haag HS (NL),Den Haag HS (NL) -- Bruxelles-Nord
25954,679,716,84____:007::8400280:8821105:4:1600:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord
25955,679,716,88____:007::8821105:8812005:22:1723:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord
25956,679,716,84____:007::8400280:8821105:4:2000:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord


Old version:

Filter the trips that are part of the allowed routes
allowed_route_type = {'IC', 'L', 'P'}
trip_route_short_name = pd.merge(trips, routes[['route_id', 'route_short_name']], on='route_id')
filtered_trips = trip_route_short_name[(trip_route_short_name['route_short_name'].isin(allowed_route_type)) | (trip_route_short_name['route_short_name'].str.startswith('S'))]
filtered_trips = filtered_trips.drop(columns=['route_short_name'])

# Exploratory data analysis with the Belgian railway data

In [19]:
'''To calculate the number of unique route_ids before removing the routes with a route_short_name that does not begin with an S and is not 'IC', 'L', or 'P'.'''
initial_set_routes = {r for r in routes['route_id']}
len(initial_set_routes)

679

In [20]:
'''To calculate the number of unique route_ids after removing the routes with a route_short_name that does not begin with an S and is not 'IC', 'L', or 'P'.'''
set_routes = {r for r in filtered_trips['route_id']}
len(set_routes)

555

In [21]:
'''To calculate the number of stations in the stops_cleaned dataset'''
set_stations = {s for s in stops_cleaned['stop_id']}
len(set_stations)

611

# Research question 1: How similar are railway systems in terms of their global network metrics?

## For this a comparison of different railway systems in a L-space network representation is performed.

##### Remember that in the L-space representation the nodes denote the stations and there exist an edge between two stations when these stations are consecutively serviced by at least one route​.

In [22]:
'''To merge a selection of the stops_cleaned dataset with a selection of the stop_times dataset'''
stops_cleaned_stop_times_merge = pd.merge(stop_times[['trip_id','arrival_time', 'departure_time','stop_id','stop_sequence']], stops_cleaned[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on='stop_id')
stops_cleaned_stop_times_merge

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,88____:007::8841673:8841004:6:559:20201212,05:35:00,05:35:00,8841673,1,Liers,50.69818,5.566829
1,88____:007::8841673:8845005:18:723:20200520,05:35:00,05:35:00,8841673,1,Liers,50.69818,5.566829
2,88____:007::8841673:8841004:6:559:20201212:1,05:35:00,05:35:00,8841673,1,Liers,50.69818,5.566829
3,88____:007::8841673:8841004:6:559:20201129,05:35:00,05:35:00,8841673,1,Liers,50.69818,5.566829
4,88____:007::8841673:8841004:6:559:20201211,05:35:00,05:35:00,8841673,1,Liers,50.69818,5.566829
...,...,...,...,...,...,...,...,...
344417,84____:007::8400530:8400058:4:2205:20200705,21:22:00,21:25:00,8400280,25,Den Haag HS (NL),52.08333,4.300000
344418,84____:007::8400280:8400530:2:1858:20201211,18:39:00,18:39:00,8400280,1,Den Haag HS (NL),52.08333,4.300000
344419,84____:007::8400280:8821105:4:2000:20201212,18:39:00,18:39:00,8400280,1,Den Haag HS (NL),52.08333,4.300000
344420,84____:007::8400280:8821105:4:2000:20200517,18:39:00,18:39:00,8400280,1,Den Haag HS (NL),52.08333,4.300000


In [23]:
'''To merge a selection of the stops_cleaned_stop_times_merge dataset with the filtered_trips dataset.'''
stops_cleaned_stop_times_trips_merge = pd.merge(filtered_trips, stops_cleaned_stop_times_merge, on='trip_id', how = 'left')
stops_cleaned_stop_times_trips_merge

,route_id,service_id,trip_id,trip_headsign,route_long_name,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers,05:35:00,05:35:00,8841673,1,Liers,50.69818,5.566829
1,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers,05:38:00,05:39:00,8841665,2,Milmort,50.69246,5.600094
2,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers,05:44:00,05:45:00,8841608,3,Herstal,50.66091,5.622762
3,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers,05:50:00,05:51:00,8841525,4,Liège-Saint-Lambert,50.64635,5.570451
4,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers,05:54:00,05:55:00,8841558,5,Liège-Carré,50.64030,5.561130
...,...,...,...,...,...,...,...,...,...,...,...,...
321418,679,716,88____:007::8821105:8812005:22:2123:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,21:03:00,21:03:00,8811262,20,Erps-Kwerps,50.89651,4.585000
321419,679,716,88____:007::8821105:8812005:22:2123:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,21:01:00,21:01:00,8811270,19,Veltem,50.90052,4.633520
321420,679,716,88____:007::8821105:8812005:22:2123:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,20:59:00,20:59:00,8811288,18,Herent,50.90353,4.672190
321421,679,716,88____:007::8821105:8812005:22:2123:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,20:11:00,20:11:00,8821063,5,Anvers-Luchtbal,51.24413,4.425033


Old version:

stops_cleaned_stop_times_merge = pd.merge(stops_cleaned[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], stop_times, on='stop_id')
stops_cleaned_stop_times_merge

stops_cleaned_stop_times_trips_merge = pd.merge(stops_cleaned_stop_times_merge, filtered_trips, on='trip_id', how = 'right')
stops_cleaned_stop_times_trips_merge

In [24]:
'''To create a route_sequence dataset that gives for each trip_id that belongs to a route the sequence of stations served'''
route_sequence = stops_cleaned_stop_times_trips_merge.groupby(['route_id','route_long_name','trip_headsign','trip_id','stop_sequence'], as_index=False)[['stop_name', 'stop_lat', 'stop_lon']].last()
route_sequence

,route_id,route_long_name,trip_headsign,trip_id,stop_sequence,stop_name,stop_lat,stop_lon
0,1,Liège-Guillemins -- Liers,Liers,88____:007::8841004:8841673:6:1057:20200524,1,Liège-Guillemins,50.62455,5.566698
1,1,Liège-Guillemins -- Liers,Liers,88____:007::8841004:8841673:6:1057:20200524,2,Liège-Carré,50.64030,5.561130
2,1,Liège-Guillemins -- Liers,Liers,88____:007::8841004:8841673:6:1057:20200524,3,Liège-Saint-Lambert,50.64635,5.570451
3,1,Liège-Guillemins -- Liers,Liers,88____:007::8841004:8841673:6:1057:20200524,4,Herstal,50.66091,5.622762
4,1,Liège-Guillemins -- Liers,Liers,88____:007::8841004:8841673:6:1057:20200524,5,Milmort,50.69246,5.600094
...,...,...,...,...,...,...,...,...
321418,679,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:818:20200705,19,Anvers-Berchem,51.19923,4.432219
321419,679,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:818:20200705,20,Anvers-Central,51.21720,4.421098
321420,679,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:818:20200705,21,Anvers-Luchtbal,51.24413,4.425033
321421,679,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:818:20200705,22,Noorderkempen (Brecht),51.35683,4.632200


Old version with the same result:

'''Per route een verduidelijking van de volgorde waarin de stations die tot die route behoren, bezocht worden'''
route_stop_sequences = stops_cleaned_stop_times_trips_merge.groupby(['route_id','trip_headsign','trip_id','stop_sequence', 'stop_name', 'stop_lat', 'stop_lon'], as_index=False)[['route_id','trip_headsign', 'trip_id', 'stop_sequence', 'stop_name', 'stop_lat', 'stop_lon']].last()
route_stop_sequences

### code voor trips_hash => too long so it is read from a csv

'''To calculate the hash value for the stop sequence of each trip_id'''

#To copy the filtered_trips dataset
trips_hash = filtered_trips.copy()

#To create a column called hash that contains NaN values
trips_hash['hash'] = np.nan

#For each trip_id in filtered_trips, the stop_sequence that gets calculated is the subset of the stop_time dataset for that trip_id. 
#The tuple that results from the stop_id column of this subset dataset contains all the stop_ids that get served by this trip_id. 
#The hash value of the tuple of the stop_id column is calculated and is placed in the hash column of the trip_id in the filtered_trips dataset

trips_hash['hash_inverse'] = np.nan
for trip in filtered_trips['trip_id'].unique():
    stop_sequence = stop_times[stop_times['trip_id'] == trip].sort_values(by = 'stop_sequence')
    trips_hash.loc[trips_hash['trip_id'] == trip, 'hash'] = hash(tuple(stop_sequence['stop_id']))
    trips_hash.loc[trips_hash['trip_id'] == trip, 'hash_inverse'] = hash(tuple(list(stop_sequence['stop_id'])[::-1]))
trips_hash
trips_hash.to_csv(r'/Users/pol/Desktop/CSV_export/trips_hash.csv', index = False, header=True, encoding='utf-8-sig')

In [25]:
trips_hash = pd.read_csv(datalink + "trips_hash.csv", sep=",")

In [26]:
trips_hash

,route_id,service_id,trip_id,trip_headsign,route_long_name,hash,hash_inverse
0,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18
1,1,48,88____:007::8841673:8841004:6:528:20200524,Liège-Guillemins,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18
2,1,48,88____:007::8841004:8841673:6:857:20200524,Liers,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18
3,1,48,88____:007::8841673:8841004:6:728:20200524,Liège-Guillemins,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18
4,1,48,88____:007::8841004:8841673:6:1057:20200524,Liers,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18
...,...,...,...,...,...,...,...
22113,679,716,84____:007::8400131:8400280:3:1720:20200705,Den Haag HS (NL),Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18
22114,679,716,84____:007::8400280:8821105:4:1600:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18
22115,679,716,88____:007::8821105:8812005:22:1723:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,8.867455e+18,-2.368845e+18
22116,679,716,84____:007::8400280:8821105:4:2000:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18


In [27]:
# To groupby the trip_id and to order the stop_sequence in an ascending order
# I did this because otherwise different hash values could correspond to a same stop_sequence (since the stop_sequences of some
# routes are initially in descending order while other stop_sequences are in ascending order)

trip_stop_sequence_sorted = stops_cleaned_stop_times_merge.groupby(['trip_id'], as_index=False).apply(lambda x: x.sort_values('stop_sequence'))
trip_stop_sequence_sorted

trip_id arrival_time  \
0     341115  80____:046::8015345:8849023:2:1011:20200419     10:04:00   
      340961  80____:046::8015345:8849023:2:1011:20200419     10:11:00   
1     341114  80____:046::8015345:8849023:2:1011:20200430     10:04:00   
      340959  80____:046::8015345:8849023:2:1011:20200430     10:11:00   
2     341112  80____:046::8015345:8849023:2:1011:20200510     10:04:00   
...                                                   ...          ...   
25957 11794    88____:L73::8843208:8841004:5:834:20201211     08:15:00   
      201716   88____:L73::8843208:8841004:5:834:20201211     08:21:00   
      201609   88____:L73::8843208:8841004:5:834:20201211     08:25:00   
      201502   88____:L73::8843208:8841004:5:834:20201211     08:28:00   
      6445     88____:L73::8843208:8841004:5:834:20201211     08:34:00   

             departure_time  stop_id  stop_sequence             stop_name  \
0     341115       10:04:00  8015345              1       Aachen Hbf (DE)   
      340961       10:11:00  8849023              2  Hergenrath-Frontière   
1     341114       10:04:00  8015345              1       Aachen Hbf (DE)   
      340959       10:11:00  8849023              2  Hergenrath-Frontière   
2     341112       10:04:00  8015345              1       Aachen Hbf (DE)   
...                     ...      ...            ...                   ...   
25957 11794        08:15:00  8843208              1        Flémalle-Haute   
      201716       08:22:00  8843067              2               Seraing   
      201609       08:26:00  8843174              3                Ougrée   
      201502       08:28:00  8847258              4              Y.RENORY   
      6445         08:36:00  8841004              5      Liège-Guillemins   

              stop_lat  stop_lon  
0     341115  50.77083  6.105277  
      340961  50.71896  6.041269  
1     341114  50.77083  6.105277  
      340959  50.71896  6.041269  
2     341112  50.77083  6.105277  
...                ...       ...  
25957 11794   50.59531  5.457661  
      201716  50.60890  5.513777  
      201609  50.60540  5.536972  
      201502  50.60485  5.557880  
      6445    50.62455  5.566698  

[344422 rows x 8 columns]

In [28]:
# To put the stop_names per trip_id in a list
'''sert à rien, le faire dans l etape hash'''
trip_stop_sequence = trip_stop_sequence_sorted.groupby('trip_id')['stop_name'].apply(lambda group_series: group_series.tolist()).reset_index()
trip_stop_sequence.rename(columns={'stop_name':'stop_sequence'}, inplace=True)
trip_stop_sequence

,trip_id,stop_sequence
0,80____:046::8015345:8849023:2:1011:20200419,"[Aachen Hbf (DE), Hergenrath-Frontière]"
1,80____:046::8015345:8849023:2:1011:20200430,"[Aachen Hbf (DE), Hergenrath-Frontière]"
2,80____:046::8015345:8849023:2:1011:20200510,"[Aachen Hbf (DE), Hergenrath-Frontière]"
3,80____:046::8015345:8849023:2:1011:20201211,"[Aachen Hbf (DE), Hergenrath-Frontière]"
4,80____:046::8015345:8849023:2:1011:20201212,"[Aachen Hbf (DE), Hergenrath-Frontière]"
...,...,...
25953,88____:L73::8841400:8841004:9:923:20201211,"[Waremme, Bléret, Remicourt, Momalle, Fexhe-Le..."
25954,88____:L73::8843208:8841004:5:1634:20201211,"[Flémalle-Haute, Seraing, Ougrée, Y.RENORY, Li..."
25955,88____:L73::8843208:8841004:5:1734:20201211,"[Flémalle-Haute, Seraing, Ougrée, Y.RENORY, Li..."
25956,88____:L73::8843208:8841004:5:734:20201211,"[Flémalle-Haute, Seraing, Ougrée, Y.RENORY, Li..."


In [29]:
# To add the stop_sequence of stations to the trips_hash dataset in the trips_hash_stop_sequence dataset by joining on trip_id
trips_hash_stop_sequence = pd.merge(trips_hash, trip_stop_sequence, on='trip_id', how='left')

# To put the columns in a more logical order
trips_hash_stop_sequence = trips_hash_stop_sequence[['route_id', 'route_long_name','service_id','trip_headsign','trip_id','hash', 'hash_inverse','stop_sequence']]
trips_hash_stop_sequence

,route_id,route_long_name,service_id,trip_headsign,trip_id,hash,hash_inverse,stop_sequence
0,1,Liège-Guillemins -- Liers,1,Namur,88____:007::8841673:8841004:6:559:20201212,-4.288363e+18,-6.465769e+18,"[Liers, Milmort, Herstal, Liège-Saint-Lambert,..."
1,1,Liège-Guillemins -- Liers,48,Liège-Guillemins,88____:007::8841673:8841004:6:528:20200524,-4.288363e+18,-6.465769e+18,"[Liers, Milmort, Herstal, Liège-Saint-Lambert,..."
2,1,Liège-Guillemins -- Liers,48,Liers,88____:007::8841004:8841673:6:857:20200524,-6.465769e+18,-4.288363e+18,"[Liège-Guillemins, Liège-Carré, Liège-Saint-La..."
3,1,Liège-Guillemins -- Liers,48,Liège-Guillemins,88____:007::8841673:8841004:6:728:20200524,-4.288363e+18,-6.465769e+18,"[Liers, Milmort, Herstal, Liège-Saint-Lambert,..."
4,1,Liège-Guillemins -- Liers,48,Liers,88____:007::8841004:8841673:6:1057:20200524,-6.465769e+18,-4.288363e+18,"[Liège-Guillemins, Liège-Carré, Liège-Saint-La..."
...,...,...,...,...,...,...,...,...
22113,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Den Haag HS (NL),84____:007::8400131:8400280:3:1720:20200705,-4.809365e+18,5.255191e+18,"[BREDA (NL), Rotterdam CS (NL), Den Haag HS (NL)]"
22114,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Bruxelles-Nord,84____:007::8400280:8821105:4:1600:20200705,-3.326243e+18,8.239505e+18,"[Den Haag HS (NL), Rotterdam CS (NL), BREDA (N..."
22115,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Bruxelles-Nord,88____:007::8821105:8812005:22:1723:20200705,8.867455e+18,-2.368845e+18,"[Noorderkempen (Brecht), Anvers-Luchtbal, Anve..."
22116,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Bruxelles-Nord,84____:007::8400280:8821105:4:2000:20200705,-3.326243e+18,8.239505e+18,"[Den Haag HS (NL), Rotterdam CS (NL), BREDA (N..."


In [30]:
# To count the number of dates for each service_id
service_id_df = calendar_dates.groupby(['service_id'])[['service_id']].count().rename(columns={'service_id':'count_service_id'}).reset_index()
service_id_df

,service_id,count_service_id
0,0,240
1,1,20
2,2,13
3,12,7
4,13,3
...,...,...
3831,5760,4
3832,5761,12
3833,5762,3
3834,5763,3


In [31]:
#NEW
#regroup the days per service id in a set
service_id_dates = calendar_dates.groupby('service_id')['date'].apply(lambda group_series: set(group_series.tolist())).reset_index()
service_id_dates.rename(columns={'date':'dates'}, inplace=True)
service_id_dates = service_id_dates.merge(service_id_df, on='service_id', how='left')
service_id_dates

,service_id,dates,count_service_id
0,0,"{20201014, 20201116, 20201001, 20201002, 20201...",240
1,1,"{20201107, 20201114, 20201121, 20200613, 20201...",20
2,2,"{20200512, 20200513, 20200514, 20200515, 20200...",13
3,12,"{20200516, 20200517, 20200501, 20200502, 20200...",7
4,13,"{20200501, 20200502, 20200503}",3
...,...,...,...
3831,5760,"{20200502, 20200501, 20200430, 20200503}",4
3832,5761,"{20200418, 20200419, 20200420, 20200421, 20200...",12
3833,5762,"{20200425, 20200426, 20200503}",3
3834,5763,"{20200426, 20200501, 20200503}",3


In [32]:
# To put the different trip_ids in a list after joining on (route_id, route_long_name, hash and service_id)
route_hash_freq = trips_hash_stop_sequence.groupby(['route_id','route_long_name','hash', 'hash_inverse', 'service_id'])['trip_id'].apply(lambda group_series: group_series.tolist()).reset_index()
route_hash_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8..."
1,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525]
2,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212]
3,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8..."
4,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212]
...,...,...,...,...,...,...
6325,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507]
6326,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8..."
6327,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ..."
6328,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ..."


In [33]:
#CHANGED
# To add the sequence of stops to the route_hash_freq dataset
route_hash_freq = pd.merge(route_hash_freq, trips_hash_stop_sequence[['route_id','hash', 'hash_inverse', 'service_id','stop_sequence']], on=['route_id', 'hash', 'hash_inverse', 'service_id'], how='left')
route_hash_freq = route_hash_freq.drop_duplicates( subset = ['route_id', 'hash', 'service_id'], keep = 'first')

route_hash_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8...","[Liège-Guillemins, Liège-Carré, Liège-Saint-La..."
8,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525],"[Liège-Guillemins, Liège-Carré, Liège-Saint-La..."
9,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212],"[Liers, Milmort, Herstal, Liège-Saint-Lambert,..."
10,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8...","[Liers, Milmort, Herstal, Liège-Saint-Lambert,..."
19,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212],"[Liège-Guillemins, Sclessin, Pont-de-Seraing, ..."
...,...,...,...,...,...,...,...
22105,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507],"[Rotterdam CS (NL), Den Haag HS (NL)]"
22106,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8...","[BREDA (NL), Rotterdam CS (NL), Den Haag HS (NL)]"
22109,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ...","[Den Haag HS (NL), Rotterdam CS (NL), BREDA (N..."
22112,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ...","[Bruxelles-Nord, Schaerbeek, Brussels Airport-..."


In [34]:
# To calculate the number of trip ids in the list of trip_ids and to add it as a new column
number_trip_ids = []
for list_trip_ids in route_hash_freq['trip_id']:
    count = len(list_trip_ids)
    number_trip_ids.append(count)
route_hash_freq['number_trip_ids'] = number_trip_ids

route_hash_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence,number_trip_ids
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8...","[Liège-Guillemins, Liège-Carré, Liège-Saint-La...",8
8,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525],"[Liège-Guillemins, Liège-Carré, Liège-Saint-La...",1
9,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212],"[Liers, Milmort, Herstal, Liège-Saint-Lambert,...",1
10,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8...","[Liers, Milmort, Herstal, Liège-Saint-Lambert,...",9
19,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212],"[Liège-Guillemins, Sclessin, Pont-de-Seraing, ...",1
...,...,...,...,...,...,...,...,...
22105,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507],"[Rotterdam CS (NL), Den Haag HS (NL)]",1
22106,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8...","[BREDA (NL), Rotterdam CS (NL), Den Haag HS (NL)]",3
22109,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ...","[Den Haag HS (NL), Rotterdam CS (NL), BREDA (N...",3
22112,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ...","[Bruxelles-Nord, Schaerbeek, Brussels Airport-...",3


In [35]:
#CHANGED
# To merge the route_hash_freq df with the service_id_df to get the sets of dates corresponding
route_hash_service_freq = pd.merge(route_hash_freq, service_id_dates, on='service_id', how='left')
route_hash_service_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence,number_trip_ids,dates,count_service_id
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8...","[Liège-Guillemins, Liège-Carré, Liège-Saint-La...",8,"{20200523, 20200524}",2
1,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525],"[Liège-Guillemins, Liège-Carré, Liège-Saint-La...",1,"{20200524, 20200525}",2
2,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212],"[Liers, Milmort, Herstal, Liège-Saint-Lambert,...",1,"{20201107, 20201114, 20201121, 20200613, 20201...",20
3,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8...","[Liers, Milmort, Herstal, Liège-Saint-Lambert,...",9,"{20200523, 20200524}",2
4,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212],"[Liège-Guillemins, Sclessin, Pont-de-Seraing, ...",1,"{20201107, 20201114, 20201121, 20200613, 20201...",20
...,...,...,...,...,...,...,...,...,...,...
6325,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507],"[Rotterdam CS (NL), Den Haag HS (NL)]",1,"{20200504, 20200507}",2
6326,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8...","[BREDA (NL), Rotterdam CS (NL), Den Haag HS (NL)]",3,"{20200704, 20200705}",2
6327,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ...","[Den Haag HS (NL), Rotterdam CS (NL), BREDA (N...",3,"{20200704, 20200705}",2
6328,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ...","[Bruxelles-Nord, Schaerbeek, Brussels Airport-...",3,"{20200704, 20200705}",2


In [36]:
route_hash_service_freq.to_csv(r'/Users/pol/Desktop/CSV_export/export_route_serv_freq.csv', index = False, header=True, encoding='utf-8-sig')

In [102]:
'''Get the distinct stop sequences for creating the possible roads combinations later on'''
distinct_stop_sequences = route_hash_service_freq.drop_duplicates(subset = ["route_id", 'hash'])[['route_id','hash','route_long_name','stop_sequence']]
#distinct_stop_sequences = distinct_stop_sequences[distinct_stop_sequences['route_id'] == 244]
distinct_stop_sequences

,route_id,hash,route_long_name,stop_sequence
0,1,-6.465769e+18,Liège-Guillemins -- Liers,"[Liège-Guillemins, Liège-Carré, Liège-Saint-La..."
2,1,-4.288363e+18,Liège-Guillemins -- Liers,"[Liers, Milmort, Herstal, Liège-Saint-Lambert,..."
4,2,-6.464971e+18,Liège-Guillemins -- Namur,"[Liège-Guillemins, Sclessin, Pont-de-Seraing, ..."
9,2,-5.388870e+18,Liège-Guillemins -- Namur,"[Liège-Guillemins, Sclessin, Pont-de-Seraing, ..."
11,2,-4.718380e+18,Liège-Guillemins -- Namur,"[Flémalle-Haute, Leman, Flémalle-Grande, Jemep..."
...,...,...,...,...
6320,678,8.032946e+18,Den Haag HS (NL) -- Bruxelles-Midi,"[Rotterdam CS (NL), Den Haag HS (NL)]"
6326,679,-4.809365e+18,Den Haag HS (NL) -- Bruxelles-Nord,"[BREDA (NL), Rotterdam CS (NL), Den Haag HS (NL)]"
6327,679,-3.326243e+18,Den Haag HS (NL) -- Bruxelles-Nord,"[Den Haag HS (NL), Rotterdam CS (NL), BREDA (N..."
6328,679,2.556917e+18,Den Haag HS (NL) -- Bruxelles-Nord,"[Bruxelles-Nord, Schaerbeek, Brussels Airport-..."


In [38]:
distinct_stop_sequences.to_csv(r'/Users/pol/Desktop/CSV_export/export_route_XXX.csv', index = False, header=True, encoding='utf-8-sig')

In [39]:
#Test to keep because interesting
selection = ['Namur', 'Tournai']
mask = distinct_stop_sequences['stop_sequence'].apply(lambda x: any(item for item in selection if (item == x[-1]) or (item == x[0])))
distinct_stop_sequences[mask]

,route_id,hash,route_long_name,stop_sequence


In [117]:
'''creates all the sequences of routes possible to reconstruct the real route'''
import copy
#create an empty df for the process of route creation
route_creation  = pd.DataFrame()
for route_id in distinct_stop_sequences['route_id'].unique():
    routes_with_route_id = distinct_stop_sequences[distinct_stop_sequences['route_id'] == route_id].copy()
    route_long_name = routes_with_route_id.iloc[0]['route_long_name'].split(' -- ')
    #create a df where only the routes that have an end stop at their beginning
    route_creation_route_id = routes_with_route_id[routes_with_route_id['stop_sequence'].apply(lambda x: any(item for item in route_long_name if item == x[0]))][['route_id','hash', 'stop_sequence']].copy()
    #make the hash column as a column of lists
    route_creation_route_id['hash'] = route_creation_route_id['hash'].apply(lambda x: [x])
    route_creatio_route_idn = route_creation_route_id.reset_index(drop=True)
    #to stop the while loop when all the routes are complete in the df
    complete_routes = 0
    while complete_routes < len(route_creation_route_id.index):
        #use a deepcopy to not impact the iterrows for loop
        route_creation_deep_copy = copy.deepcopy(route_creation_route_id)
        for index_original, route_part in route_creation_deep_copy.iterrows():
            print(index_original)
            #if the trip is not complete (contains begin and end), add some other parts
            if not(set(route_long_name).issubset(route_part['stop_sequence'])):
                #create a dataframe of the possible extentions for each route_part
                #here checks if the extention is the next part of the route and also that no other station are repeated (otherwise it might cause an infinite loop)
                possible_extentions = routes_with_route_id[routes_with_route_id['stop_sequence'].apply(lambda x: any(item for item in [route_part['stop_sequence'][-1]] if (item == x[0]) and not(set(x[1:]) & set(route_part['stop_sequence']))))].copy()
                #checks whether it is possible to extend the route_part
                if not (possible_extentions.empty):
                    print('route creation before extentions added ------')
                    print(route_creation)
                    print('-------')
                    print('extentions ----')
                    print(possible_extentions)
                    print('------')
                    #if so, extend it with every single possibilities
                    for index_extention, possible_extention in possible_extentions.iterrows():
                        #must create a deepcopy, otherwise the orignal hash list will change as well (mutable)
                        updated_hash = copy.deepcopy(route_part['hash'])
                        updated_hash.append(possible_extention['hash'])
                        updated_route_sequence = route_part['stop_sequence'] + possible_extention['stop_sequence'][1:]
                        #route_creation = route_creation.append({'route_id': route_id, 'hash': updated_hash, 'stop_sequence': route_part['stop_sequence'] + possible_extention['stop_sequence'] } ,ignore_index=True)
                        route_creation_route_id.loc[max(route_creation_route_id.index)+1] = [route_id, updated_hash, updated_route_sequence] 
                #then delete the route with the index (see loop here above)
                print('route creation after extentions added ------')
                print(route_creation_route_id)
                print('-------')
                route_creation_route_id = route_creation_route_id.drop(index = index_original)
                print('route creation after drop made ------')
                print(route_creation_route_id)
                print('-------')
            #when the route is complete    
            else:
                complete_routes += 1     
    route_creation = route_creation.append(route_creation_route_id, ignore_index = True)
            #if it is possible, just delete the path or recreate a route (to discuss)
            #then the routes that are not subset of those routes need to be created as well

0
2
4
9
route creation before extentions added ------
   route_id                      hash  \
0         1  [-6.465769007291632e+18]   
1         1  [-4.288362932254336e+18]   

                                       stop_sequence  
0  [Liège-Guillemins, Liège-Carré, Liège-Saint-La...  
1  [Liers, Milmort, Herstal, Liège-Saint-Lambert,...  
-------
extentions ----
    route_id          hash            route_long_name  \
29         2  7.223954e+18  Liège-Guillemins -- Namur   

                                        stop_sequence  
29  [Andenne, Château-de-Seilles, Sclaigneaux, Nam...  
------
route creation after extentions added ------
    route_id                                             hash  \
4          2                         [-6.464970586350677e+18]   
9          2                         [-5.388870276730761e+18]   
13         2                        [-4.5573246640718126e+18]   
16         2                         [-3.538425721227366e+18]   
21         2                 

50  [Liège-Guillemins, Liège-Carré, Liège-Saint-La...  
------
route creation after extentions added ------
     route_id                                               hash  \
156         3    [-3.3047935184970286e+18, -3.4679091469873e+18]   
158         3   [-3.3047935184970286e+18, 4.268900537161168e+18]   
159         3   [-3.858761614253005e+17, 2.3416502157266227e+18]   
160         3       [6.89496055130595e+18, 8.68260881516486e+18]   
161         3    [7.353588791640738e+18, -6.930515448429063e+18]   
162         3       [7.353588791640738e+18, -2540391171726936.0]   
163         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
164         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
165         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
166         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
167         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
168         3  [-4.288362932254336e+18, 5.206382830626538e+17...   
169     

     route_id                                               hash  \
156         3    [-3.3047935184970286e+18, -3.4679091469873e+18]   
162         3       [7.353588791640738e+18, -2540391171726936.0]   
165         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
171         3  [-4.288362932254336e+18, 1.7001387242160358e+1...   
173         3  [-3.3047935184970286e+18, -6.226379545034256e+...   
174         3  [-3.3047935184970286e+18, 5.762163817894722e+1...   
175         3  [-3.3047935184970286e+18, 5.762163817894722e+1...   
176         3  [-3.3047935184970286e+18, 4.268900537161168e+1...   
177         3  [-3.858761614253005e+17, 2.3416502157266227e+1...   
178         3  [-3.858761614253005e+17, 2.3416502157266227e+1...   
179         3  [6.89496055130595e+18, 8.68260881516486e+18, -...   
180         3  [6.89496055130595e+18, 8.68260881516486e+18, -...   
181         3  [6.89496055130595e+18, 8.68260881516486e+18, -...   
182         3  [6.89496055130595e+18, 8.68260881

210  [Luxembourg (LU), Pfaffenthal-Kirchberg (LU), ...  
-------
route creation after drop made ------
     route_id                                               hash  \
156         3    [-3.3047935184970286e+18, -3.4679091469873e+18]   
162         3       [7.353588791640738e+18, -2540391171726936.0]   
165         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
171         3  [-4.288362932254336e+18, 1.7001387242160358e+1...   
175         3  [-3.3047935184970286e+18, 5.762163817894722e+1...   
176         3  [-3.3047935184970286e+18, 4.268900537161168e+1...   
178         3  [-3.858761614253005e+17, 2.3416502157266227e+1...   
181         3  [6.89496055130595e+18, 8.68260881516486e+18, -...   
184         3  [7.353588791640738e+18, -6.930515448429063e+18...   
188         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
189         3  [-8.108149592008595e+18, 7.437881316535453e+18...   
192         3  [-4.288362932254336e+18, 5.206382830626538e+17...   
193         3

268  [Ettelbruck(LU), Michelau (LU), Kautenbach (LU...  
-------
route creation after drop made ------
     route_id                                              hash  \
267         9                            [8.68260881516486e+18]   
268         9  [-2.6900359395340887e+18, 7.437881316535453e+18]   

                                         stop_sequence  
267  [Ettelbruck(LU), Michelau (LU), Kautenbach (LU...  
268  [Ettelbruck(LU), Michelau (LU), Kautenbach (LU...  
-------
267
route creation before extentions added ------
    route_id                                               hash  \
0          1                           [-6.465769007291632e+18]   
1          1                           [-4.288362932254336e+18]   
2          2                           [-6.464970586350677e+18]   
3          2                           [-3.538425721227366e+18]   
4          2    [-5.388870276730761e+18, 7.223954040765058e+18]   
5          2  [-4.5573246640718126e+18, -4.718380454349116e+18] 

...

5981
5983
5987
5989
5990
5992
5993
route creation before extentions added ------
      route_id                                             hash  \
0            1                         [-6.465769007291632e+18]   
1            1                         [-4.288362932254336e+18]   
2            2                         [-6.464970586350677e+18]   
3            2                         [-3.538425721227366e+18]   
4            2  [-5.388870276730761e+18, 7.223954040765058e+18]   
...        ...                                              ...   
1493       621                         [-1.319588714196605e+18]   
1494       622                         [1.0213597934065427e+17]   
1495       622                          [8.101970481505922e+18]   
1496       623                         [-6.395340060003871e+18]   
1497       623                         [-5.325011278806588e+17]   

                                          stop_sequence  
0     [Liège-Guillemins, Liège-Carré, Liège-Saint-La... 

6120  [Courtrai, Harelbeke, Waregem, Deinze, De Pint...  
-------
route creation after drop made ------
      route_id                                               hash  \
6116       656                           [4.3721943049872737e+18]   
6117       656  [-4.4458925823372943e+18, -6.089526838321164e+18]   
6118       656  [-4.4458925823372943e+18, -5.789241225446191e+18]   
6119       656    [-2.460257712409705e+18, 8.266136521667648e+17]   
6120       656   [-4.465845616961985e+17, -8.779084995067235e+18]   

                                          stop_sequence  
6116  [Courtrai, Harelbeke, Waregem, Deinze, De Pint...  
6117  [Termonde, Oudegem, Schoonaarde, Wichelen, Sch...  
6118  [Termonde, Oudegem, Schoonaarde, Wichelen, Sch...  
6119  [Courtrai, Harelbeke, Waregem, Deinze, De Pint...  
6120  [Courtrai, Harelbeke, Waregem, Deinze, De Pint...  
-------
6116
6116
6117
6118
route creation before extentions added ------
      route_id                                             

[1579 rows x 3 columns]
-------
extentions ----
      route_id          hash                      route_long_name  \
6188       673 -5.482836e+18  Amsterdam Cs (NL) -- Bruxelles-Midi   
6242       673  4.803089e+18  Amsterdam Cs (NL) -- Bruxelles-Midi   
6249       673  6.313648e+18  Amsterdam Cs (NL) -- Bruxelles-Midi   

                                          stop_sequence  
6188  [BREDA (NL), Rotterdam CS (NL), Den Haag HS (N...  
6242  [BREDA (NL), Rotterdam CS (NL), Schiphol (NL),...  
6249                    [BREDA (NL), Rotterdam CS (NL)]  
------
route creation after extentions added ------
      route_id                                               hash  \
6241       673                           [4.4891922201522473e+18]   
6244       673                            [5.821380762008269e+18]   
6245       673  [-9.222261791946643e+18, -1.3895296309575789e+17]   
6246       673    [-8.32433174168094e+18, -5.482835746316169e+18]   
6247       673     [-8.32433174168094e+18, 4.8

[1579 rows x 3 columns]
-------
extentions ----
      route_id          hash                      route_long_name  \
6189       673 -4.891598e+18  Amsterdam Cs (NL) -- Bruxelles-Midi   

                                          stop_sequence  
6189  [Anvers-Central, Anvers-Berchem, Mortsel-Deurn...  
------
route creation after extentions added ------
      route_id                                               hash  \
6246       673    [-8.32433174168094e+18, -5.482835746316169e+18]   
6247       673     [-8.32433174168094e+18, 4.803088650834876e+18]   
6250       673    [-8.23096153150583e+18, 2.8450235790157635e+17]   
6251       673     [-8.23096153150583e+18, 3.225997909968076e+18]   
6252       673      [-8.23096153150583e+18, 6.80384510654198e+18]   
6254       673   [-2.892405721056469e+18, 2.8450235790157635e+17]   
6255       673    [-2.892405721056469e+18, 3.225997909968076e+18]   
6256       673     [-2.892405721056469e+18, 6.80384510654198e+18]   
6258       673   [4.4891

route creation before extentions added ------
      route_id                                               hash  \
0            1                           [-6.465769007291632e+18]   
1            1                           [-4.288362932254336e+18]   
2            2                           [-6.464970586350677e+18]   
3            2                           [-3.538425721227366e+18]   
4            2    [-5.388870276730761e+18, 7.223954040765058e+18]   
...        ...                                                ...   
1612       675  [1.470269866367346e+18, -3.156733466808462e+18...   
1613       675  [2.556917391222364e+18, 6.313648002564415e+18,...   
1614       675  [1.470269866367346e+18, -3.156733466808462e+18...   
1615       676                            [4.092876304659129e+18]   
1616       676                            [7.619387798867893e+18]   

                                          stop_sequence  
0     [Liège-Guillemins, Liège-Carré, Liège-Saint-La...  
1     [Li

In [124]:
print(route_creation.loc[1600])

route_id                                                       673
hash             [5.821380762008269e+18, -1.3895296309575789e+1...
stop_sequence    [Amsterdam Cs (NL), Schiphol (NL), Rotterdam C...
Name: 1600, dtype: object


In [122]:
route_creation.to_csv(r'/Users/pol/Desktop/CSV_export/export_route_creation1.csv', index = False, header=True, encoding='utf-8-sig')

In [125]:
'''after the creation of the routes => get the frequency of the route'''
route_creation['frequency'] = 0
for index_sequence, sequence in route_creation[['route_id','hash']].iterrows():
    sequence_frequency = 0
    set_common_service_id = set()
    #take all the trips with that route_id
    selected_route_hash_service_freq =route_hash_service_freq[route_hash_service_freq['route_id']== sequence['route_id']]
    for trip_hash in sequence['hash']:
        corresponding_service_id = set()
        #loops on the trips with that hash value and take their date
        for service_id in selected_route_hash_service_freq[selected_route_hash_service_freq['hash']== trip_hash]['service_id']:
            #union the dates with the same hash value 
            corresponding_service_id.add(service_id)
        #if the set is empty, initiate it
        if not set_common_service_id:
            set_common_service_id = corresponding_service_id
        #if the set is not empty take the intersection, to only have the dates that are common
        else:
            set_common_service_id = set_common_service_id.intersection(corresponding_service_id)
    if sequence['route_id'] == 244:
        print(sequence['hash'])
        print(set_common_service_id)
    if set_common_service_id:
        #Only select the trips with the hash value contained in the sequence and with the same route_id
        containing_hash = selected_route_hash_service_freq[selected_route_hash_service_freq['hash'].apply(lambda x: any(item for item in sequence['hash'] if x == item))]
        #loop over each date that were common during the trip
        for service_id in set_common_service_id:
            #take all the trips with the selected hashes that contain the date of the loop
            containing_service_id = containing_hash[containing_hash['service_id'].apply(lambda x: any(item for item in [service_id] if item == x))]
            #takes the out of all the daily frequency the minimum of all the trips (otherwise they won't be able to run completely, since some part do not run anymore)
            #this could potentially be changed by service id or by issubset() (checking is the smallest set is contained by the biggest one)
            containing_service_id_grouped = containing_service_id.groupby(['hash'], as_index = False)['number_trip_ids'].sum()
            #adds the minimum number of trips times the number of days in the service_id
            if sequence['route_id'] == 244:
                print('ADDITION')
                print(containing_service_id_grouped['number_trip_ids'].min() * service_id_df[service_id_df['service_id'] == service_id].iloc[0]['count_service_id'])
            sequence_frequency += containing_service_id_grouped['number_trip_ids'].min() * service_id_df[service_id_df['service_id'] == service_id].iloc[0]['count_service_id']
            if sequence['route_id'] == 244:
                print('mini trip id -------')
                print(containing_service_id_grouped['number_trip_ids'].min())
                print('-------')
                print('mini count service id ------')
                print(service_id_df[service_id_df['service_id'] == service_id].iloc[0]['count_service_id'])
                print('-------')
        if sequence['route_id'] == 244:
                print('SEQUENCE FREQ')
                print(sequence_frequency)
        route_creation.loc[index_sequence, 'frequency'] = sequence_frequency
        if sequence['route_id'] == 244:
                print('Route Place')
                print(index_sequence)
                print(route_creation.loc[index_sequence])

[-8.198854371573214e+18, -2.444960621321155e+18, -2.637981552073598e+18]
{358, 390}
ADDITION
189
mini trip id -------
7
-------
mini count service id ------
27
-------
ADDITION
588
mini trip id -------
7
-------
mini count service id ------
84
-------
SEQUENCE FREQ
777
Route Place
466
route_id                                                       244
hash             [-8.198854371573214e+18, -2.444960621321155e+1...
stop_sequence    [Lille Flandres (FR), Pont de Bois (FR), Ascq ...
frequency                                                      777
Name: 466, dtype: object
[-4.910059532324571e+18, -2.444960621321155e+18, -2.637981552073598e+18]
{358, 390}
ADDITION
27
mini trip id -------
1
-------
mini count service id ------
27
-------
ADDITION
84
mini trip id -------
1
-------
mini count service id ------
84
-------
SEQUENCE FREQ
111
Route Place
467
route_id                                                       244
hash             [-4.910059532324571e+18, -2.444960621321155e+1...
sto

In [126]:
route_creation.to_csv(r'/Users/pol/Desktop/CSV_export/export_route_creation2.csv', index = False, header=True, encoding='utf-8-sig')

In [44]:
route_creation

,route_id,hash,stop_sequence,frequency
3,9,"[8.68260881516486e+18, -3.4679091469873e+18]","[Ettelbruck(LU), Michelau (LU), Kautenbach (LU...",4
6,9,"[-2.6900359395340887e+18, 7.437881316535453e+1...","[Ettelbruck(LU), Michelau (LU), Kautenbach (LU...",4
10,9,"[8.68260881516486e+18, 4.268900537161168e+18, ...","[Ettelbruck(LU), Michelau (LU), Kautenbach (LU...",80
12,9,"[-2.6900359395340887e+18, 7.437881316535453e+1...","[Ettelbruck(LU), Michelau (LU), Kautenbach (LU...",16
13,9,"[8.68260881516486e+18, 5.762163817894722e+16, ...","[Ettelbruck(LU), Michelau (LU), Kautenbach (LU...",12
14,9,"[-2.6900359395340887e+18, 7.437881316535453e+1...","[Ettelbruck(LU), Michelau (LU), Kautenbach (LU...",12


In [45]:
#test to delete
i = 0
hello = [1, 2 ,2,  3, 4, 5]
for item in hello:
    hello.pop(i)
    i += 1
    if i == 2:
        break
print (hello)

[2, 3, 4, 5]


# Previous code


In [46]:
route_hash_service_freq.groupby('service_id')['dates'].apply(list)

service_id
1       [{20201107, 20201114, 20201121, 20200613, 2020...
2       [{20200512, 20200513, 20200514, 20200515, 2020...
12      [{20200516, 20200517, 20200501, 20200502, 2020...
13      [{20200501, 20200502, 20200503}, {20200501, 20...
14      [{20200620, 20200621, 20200910, 20200911, 2020...
                              ...                        
5760    [{20200502, 20200501, 20200430, 20200503}, {20...
5761    [{20200418, 20200419, 20200420, 20200421, 2020...
5762    [{20200425, 20200426, 20200503}, {20200425, 20...
5763    [{20200426, 20200501, 20200503}, {20200426, 20...
5764    [{20200425, 20200418, 20200502}, {20200425, 20...
Name: dates, Length: 1141, dtype: object

In [47]:
# To group the previous route_hash_service_freq df to get unique rows that contain for each route_id the stop sequences 
# and an overview of the trip_ids that belong to this route_id and travel the specific stop_sequence
route_hash_freq = route_hash_service_freq.groupby(['route_id','route_long_name','hash', 'hash_inverse', 'freq_sequence_route'], as_index=False)[['trip_id', 'stop_sequence']].first()
route_hash_freq

KeyError: 'freq_sequence_route'

In [ ]:
'''Regroup the routes that are the same (even though they are in the opposite direction)'''
route_hash_freq_max_hash = route_hash_freq.copy()
route_hash_freq_max_hash['max_hash'] = route_hash_freq[['hash', 'hash_inverse']].max(axis=1)
#create a df that sums the frequence of the trips going from opposite directions
route_hash_freq_max_hash_freq = route_hash_freq_max_hash.groupby(['route_id','max_hash'], as_index = False)[['freq_sequence_route']].sum()
#renames the max_hash column into hash so it the dataframe can be merged with route_hash_without_freq
route_hash_freq_max_hash_freq = route_hash_freq_max_hash_freq.rename(columns = {'max_hash':'hash'})
#drops the column freq_sequence_route because the one that is of interest is in route_hash_freq_max_hash_freq
route_hash_without_freq = route_hash_freq.drop(['freq_sequence_route'], axis = 1)
route_hash_freq_combined_first_merge = pd.merge(route_hash_freq_max_hash_freq, route_hash_without_freq, on=['route_id', 'hash'], how='left')
route_hash_freq_combined_first_merge  = route_hash_freq_combined_first_merge.drop(['hash_inverse'], axis = 1)
#selects one part the part of the dataset that doesn't have NaN, so we can concatenate it with the part that had NaN later
route_hash_freq_first_part = route_hash_freq_combined_first_merge[pd.notnull(route_hash_freq_combined_first_merge['trip_id'])]
#selects one part the part of the dataset that does have NaN, so we can concatenate it with the part that has no NaN.
#but first, we will need to fill those NaN values (done in the code lines behind this one)
route_hash_freq_second_part = route_hash_freq_combined_first_merge[pd.isnull(route_hash_freq_combined_first_merge['trip_id'])][['route_id', 'hash', 'freq_sequence_route']]
#renames the hash column into hash_inverse so it the dataframe can be merged with route_hash_without_freq (because it didn't work with 'hash')
route_hash_freq_second_part = route_hash_freq_second_part.rename(columns = {'hash':'hash_inverse'})
route_hash_freq_second_part = pd.merge(route_hash_freq_second_part, route_hash_without_freq, on=['route_id', 'hash_inverse'], how='left')
#the hash that is of interest in the final df will be hash and not hash_inverse
route_hash_freq_second_part  = route_hash_freq_second_part.drop(['hash_inverse'], axis = 1)
route_hash_freq_combined_not_sorted = pd.concat([route_hash_freq_first_part, route_hash_freq_second_part])
route_hash_freq_combined = route_hash_freq_combined_not_sorted.sort_values(by = ['route_id'])
route_hash_freq_combined = route_hash_freq_combined.reset_index(drop = True)
route_hash_freq_combined


# Check whether the stop sequence and hash toghether are correct

In [ ]:
route_hash_freq_combined.to_csv(r'/Users/pol/Desktop/CSV_export/export_route_hash_freq_combined.csv', index = False, header=True, encoding='utf-8-sig')

In [ ]:
import copy
from statistics import mean 
'''Recreates the routes that were cut in several parts'''
#create a route index in order to create new route_ids that are after the exisiting ones
route_index = route_hash_freq_combined['route_id'].max() + 1
#create an empty df that has the same columns as route_hash_freq_combined execpt 'hash' and 'trip_id'
full_routes = route_hash_freq_combined.copy()
full_routes = full_routes.drop(full_routes.index)
full_routes = full_routes.drop(columns=['hash', 'trip_id'])
#here for the test we just check with route 3
route_hash_freq_combined_copy = route_hash_freq_combined.copy()
#takes the most frequent stop sequence per route
route_hash_max_id = route_hash_freq_combined_copy.loc[route_hash_freq_combined_copy.groupby('route_id')['freq_sequence_route'].idxmax()].copy()
#checks whether it is the full route or not
for route_id in route_hash_max_id['route_id'].unique():
    for index, max_id_row in route_hash_max_id[route_hash_max_id['route_id'] == route_id][['route_id', 'freq_sequence_route','route_long_name', 'stop_sequence']].iterrows():
        #make a deepcopy, otherwise it impacts the previous df
        max_id_row['stop_sequence'] = copy.deepcopy(max_id_row['stop_sequence'])
        route_long_name = max_id_row['route_long_name'].split(' -- ')
        #checks whether the stop sequence contains the begin and end station
        if (route_long_name[0] in max_id_row['stop_sequence']) and (route_long_name[1] in max_id_row['stop_sequence']):
            full_routes = full_routes.append(max_id_row, ignore_index=True)
        #checks whether the stop sequence contains the begin or end station
        elif (route_long_name[0] in max_id_row['stop_sequence']) or (route_long_name[1] in max_id_row['stop_sequence']):
            #makes a list in order to add the frequences of the different trips and avg them in the end
            freq_sequences_list = [max_id_row['freq_sequence_route']]
            #loops until the stop sequence is full and stops if it is impossible to create the complete stop_sequence
            while not((route_long_name[0] in max_id_row['stop_sequence']) and (route_long_name[1] in max_id_row['stop_sequence'])):
                begin_sequence = copy.deepcopy(max_id_row['stop_sequence'])
                #loops over the other stop sequences of the same route to find the next part of the sequence
                for index2, other_stop_sequence in route_hash_freq_combined[route_hash_freq_combined['route_id'] == route_id].iterrows():
                    #checks whether the stop sequence is the continuity of the main one
                    #here we check wether the which part of the route is the end or begin                        
                    if (route_long_name[0] == max_id_row['stop_sequence'][-1]) or (route_long_name[1] in max_id_row['stop_sequence'][-1]):
                        #reverse the route if it is not the right order and as a result will comply to the next if
                        max_id_row['stop_sequence'] = max_id_row['stop_sequence'][::-1]
                    if (route_long_name[0] == max_id_row['stop_sequence'][0]) or (route_long_name[1] in max_id_row['stop_sequence'][0]):
                        #if the sequel stop is at the end of the stop sequence, reverse it first
                        if (max_id_row['stop_sequence'][-1] == other_stop_sequence['stop_sequence'][-1]) and not (set(other_stop_sequence['stop_sequence']).issubset(max_id_row['stop_sequence'])):
                            other_stop_sequence['stop_sequence'] = other_stop_sequence['stop_sequence'][::-1]
                        #here if the list is the continuity, the 
                        if max_id_row['stop_sequence'][-1] == other_stop_sequence['stop_sequence'][0] and not (set(other_stop_sequence['stop_sequence'][::-1]).issubset(max_id_row['stop_sequence'])):
                            #here we don't take the first element of the other list, since it is the same as the last of the extended list
                            max_id_row['stop_sequence'].extend(other_stop_sequence['stop_sequence'][1:])
                            freq_sequences_list.append(other_stop_sequence['freq_sequence_route'])
                    else:
                        raise ValueError('No end or begin station with with route_id: ' + str(route_id))
                        break
                    #if it is finish, break
                    if (route_long_name[0] in max_id_row['stop_sequence']) and (route_long_name[1] in max_id_row['stop_sequence']):
                        max_id_row['freq_sequence_route'] = mean(freq_sequences_list)
                        full_routes = full_routes.append(max_id_row, ignore_index=True)
                        break
                #if after the iteration the route could not be extended, stop the loop and create a new route
                if begin_sequence == max_id_row['stop_sequence']:
                    print('For route_id: ' + str(route_id) + ' the sequence can not be extended')
                    max_id_row['route_long_name'] = max_id_row['stop_sequence'][0] + ' -- ' + max_id_row['stop_sequence'][-1]
                    max_id_row['freq_sequence_route'] = mean(freq_sequences_list)
                    max_id_row['route_id'] = route_index
                    #increment the route_index so it won't be the same for the next exception
                    route_index += 1
                    full_routes = full_routes.append(max_id_row, ignore_index=True)
                    break
        #Is a complete different route than begin and end station
        else:
            print('For route_id: ' + str(route_id) + ' the route does not comply to begin and end station')
            max_id_row['route_long_name'] = max_id_row['stop_sequence'][0] + ' -- ' + max_id_row['stop_sequence'][-1]
            max_id_row['route_id'] = route_index
            #increment the route_index so it won't be the same for the next exception
            route_index += 1
            full_routes = full_routes.append(max_id_row, ignore_index=True)
            
print(full_routes)

In [ ]:
'''find the routes that are not subsets of their respective representative route'''
for route_id in full_routes['route_id']:
    if not route_hash_freq_combined[route_hash_freq_combined['route_id'] == route_id].empty:
        for index, trip in route_hash_freq_combined[route_hash_freq_combined['route_id'] == route_id].iterrows():
            if not set(trip['stop_sequence']).issubset(full_routes[full_routes['route_id'] == route_id].iloc[0]['stop_sequence']):
                print('for route_id: ' + str(route_id) + ' and hash: ' + str(trip['hash']))

In [ ]:
full_routes.to_csv(r'/Users/pol/Desktop/CSV_export/export_full_routes.csv', index = False, header=True, encoding='utf-8-sig')

# Vanaf hier start de oudere code (met frequenste route)

In [ ]:
'''To calculate the hash value for the stop sequence of each trip_id'''

#To copy the filtered_trips dataset
trips_hash = filtered_trips.copy()

#To create a column called hash that contains NaN values
trips_hash['hash'] = np.nan

#For each trip_id in filtered_trips, the stop_sequence that gets calculated is the subset of the stop_time dataset for that trip_id. 
#The tuple that results from the stop_id column of this subset dataset contains all the stop_ids that get served by this trip_id. 
#The hash value of this tuple is calculated and is placed in the hash column of the trip_id in the filtered_trips dataset

trips_hash = filtered_trips.copy()
trips_hash['hash'] = np.nan
for trip in filtered_trips['trip_id'].unique():
    stop_sequence = stop_times[stop_times['trip_id'] == trip].sort_values(by = 'stop_sequence')
    trips_hash.loc[trips_hash['trip_id'] == trip, 'hash'] = hash(tuple(stop_sequence['stop_id']))

The same version with less comments:

'''Calculates the hash value for the stop sequences per trip_id'''
trips_hash = filtered_trips.copy()
trips_hash['hash'] = np.nan
for trip in filtered_trips['trip_id'].unique():
    stop_sequence = stop_times[stop_times['trip_id'] == trip].sort_values(by = 'stop_sequence')
    trips_hash.loc[trips_hash['trip_id'] == trip, 'hash'] = hash(tuple(stop_sequence['stop_id']))

In [ ]:
trips_hash

In [ ]:
'''To create a list that contains for each route the most frequent trip belonging to that route'''
#To count the number of dates for each service_id
days_per_service_id = calendar_dates[['service_id','date']].groupby('service_id').nunique()

#To create a copy of the trips_hash dataset
trips_hash_copy= trips_hash.copy()

#To put the number of dates of each service_id in the occurence_operated_trips dataset
occurence_operated_trips = trips_hash_copy.join(days_per_service_id, on='service_id')

#To calculate the number of occurences of all the different station sequences of each route
number_operated_trips_per_route = occurence_operated_trips[['route_id','hash', 'date']].groupby(['route_id', 'hash']).sum()

#To make a boolean index where the most frequent number of occurences for a specific route_id into account has a value True
idx = number_operated_trips_per_route.groupby(['route_id'])['date'].transform(max) == number_operated_trips_per_route['date']

#To select for each route the rows that contain the most frequent number of occurences
best_trips_per_route = number_operated_trips_per_route[idx]
best_trips_per_route.reset_index(inplace=True)

#To keep for each route only the last row with the highest number of occurences
best_trip_per_route = best_trips_per_route.drop_duplicates( 
  subset = ['route_id'], 
  keep = 'last').reset_index(drop = True)

#To construct an empty list and an empty dictionary called most_frequent_trips_per_route and most_frequent_trips_per_route_dic 
most_frequent_trips_per_route = []
most_frequent_trips_per_route_dic = {}

#For each row in the best_trip_per_route dataset, the first trip_id in the trips_hash dataset that has the same route_id and hash values
#as the best_trip_per_route row is selected and placed in the most_frequent_trips_per_route list.
#The most_frequent_trips_per_route dictionary is constructed by using these trip_ids as the keys and the corresponding hash value as the 
#value of the key.

for index, row in best_trip_per_route.iterrows():
    trip_id = trips_hash[(trips_hash['route_id'] == row['route_id']) & (trips_hash['hash'] == row['hash'])]['trip_id'].iloc[0]
    most_frequent_trips_per_route.append(trip_id)
    most_frequent_trips_per_route_dic[trip_id] = row['hash']

Old version: the same but with different comments

'''Creates a list with an instance of the most frequent trip for each route'''
#######counts the number of days for each service_id
days_per_service_id = calendar_dates[['service_id','date']].groupby('service_id').nunique()
trips_hash_copy= trips_hash.copy()
#######puts the number of days with each instance of the trips
occurence_operated_trips = trips_hash_copy.join(days_per_service_id, on='service_id')
#######counts the number of occurences of each type of trip sequence for each routes
number_operated_trips_per_route = occurence_operated_trips[['route_id','hash', 'date']].groupby(['route_id', 'hash']).sum()
#######max an index that only takes the max values of the number of occurences calculated above for each route
idx = number_operated_trips_per_route.groupby(['route_id'])['date'].transform(max) == number_operated_trips_per_route['date']
#######select only the row that have the maximum value of occurences for each route
best_trips_per_route = number_operated_trips_per_route[idx]
best_trips_per_route.reset_index(inplace=True)
#######for each route, drop the rows that have the same max value (there might be more than one trip with the max number of occurence for a route)
best_trip_per_route = best_trips_per_route.drop_duplicates( 
  subset = ['route_id'], 
  keep = 'last').reset_index(drop = True)
most_frequent_trips_per_route = []
most_frequent_trips_per_route_dic = {}
#######adds in the list a trip that begins with the same characters (i.e that has the same stop frequence) and that has the same route_id
for index, row in best_trip_per_route.iterrows():
    trip_id = trips_hash[(trips_hash['route_id'] == row['route_id']) & (trips_hash['hash'] == row['hash'])]['trip_id'].iloc[0]
    most_frequent_trips_per_route.append(trip_id)
    most_frequent_trips_per_route_dic[trip_id] = row['hash']

In [ ]:
''' To create a dictionary that has the hash values of the most frequent trip_ids per route as key and all possible corresponding
trip_ids that have that hash value (and thus serve a same sequence of stops) as the value of the hash key'''

# To construct an empty dictionary duplicate_trip_sequence
duplicate_trip_sequence = {}

# To put the hash values of the trip_ids in the trips_hash dataset that correspond to the most frequent trips per route in a list
output_list = [trips_hash[trips_hash['trip_id'] == i]['hash'].iloc[0] for i in most_frequent_trips_per_route]

# To create a set of unique hash values that are hash values of at least one most frequent trip_id
duplicate_route_segments = set([x for x in output_list if output_list.count(x) > 1])
duplicate_route_segments

# By iterating over the unique hash values in duplicate_route_segments, it is checked for the trip_ids in the most_frequent_trips_per_route_dic
# dictionary that contains all most frequent trip_ids, if the corresponding hash values of these trip_ids are equal to a 
# hash value in the duplicate_route_segments set.
# All resulting hash values are put in the duplicate_trip_sequence dictionary as a key and all corresponding trip_ids that have this
# hash values are added as the corresponding value of this hash key

for duplicate_trip_hash in duplicate_route_segments:
    for trip_id in most_frequent_trips_per_route_dic:
        if most_frequent_trips_per_route_dic[trip_id] == duplicate_trip_hash:
            if duplicate_trip_hash not in duplicate_trip_sequence:
                duplicate_trip_sequence[duplicate_trip_hash] = []
            duplicate_trip_sequence[duplicate_trip_hash].append(trip_id)
            
''' To keep only the first of the two trip_ids in the duplicate_trip_sequence dictionary for each hash value'''

for duplicate_trip_hash in duplicate_trip_sequence:
    for trip_id in duplicate_trip_sequence[duplicate_trip_hash][:-1]:
        most_frequent_trips_per_route.remove(trip_id)

The same version with different comments:

''''checks if some routes have the similar sequences
####This means:
####For all the trip_id's in representative_trips, it is checked by means of the first characters of the trip_id, if the same sequence of stations is visited for multiple route_id's
'''
####puts all the trips that have the same begin characters for trip_id in a dictionnary (of the most_frequent_trips_per_route)
duplicate_trip_sequence = {}
output_list = [trips_hash[trips_hash['trip_id'] == i]['hash'].iloc[0] for i in most_frequent_trips_per_route]
duplicate_route_segments = set([x for x in output_list if output_list.count(x) > 1])
for duplicate_trip_hash in duplicate_route_segments:
    for trip_id in most_frequent_trips_per_route_dic:
        if most_frequent_trips_per_route_dic[trip_id] == duplicate_trip_hash:
            if duplicate_trip_hash not in duplicate_trip_sequence:
                duplicate_trip_sequence[duplicate_trip_hash] = []
            duplicate_trip_sequence[duplicate_trip_hash].append(trip_id)

####removes all the trips that have a same trip sequence as another one, except one per trip sequence
for duplicate_trip_hash in duplicate_trip_sequence:
    for trip_id in duplicate_trip_sequence[duplicate_trip_hash][:-1]:
        most_frequent_trips_per_route.remove(trip_id)

In [ ]:
'''Makes a set that can be used for building the edges of the graph using Networkx package
This means:
...
The df_for_edges dataset contains for each route_id one trip_id. However, there are still trip_id's with the same sequence of stations in df_for_edges
'''
#Take only one instance of the most freqeuent trips
stops_cleaned_stop_times_trips_merge_singletrip = stops_cleaned_stop_times_trips_merge[stops_cleaned_stop_times_trips_merge['trip_id'].isin(most_frequent_trips_per_route)]
#Take each stop of a trip instance once
stops_cleaned_stop_times_trips_merge_grouped_by = stops_cleaned_stop_times_trips_merge_singletrip.groupby(['route_id','trip_headsign', 'trip_id','stop_sequence', 'stop_name'], as_index=False)[[ 'stop_lat', 'stop_lon']].last()
#Creates a shifted instance of the df to use it for the final result
stops_cleaned_stop_times_trips_merge_grouped_by_shifted = stops_cleaned_stop_times_trips_merge_grouped_by.shift()
#Check if which of the rows are followed by a row with the same trip_id
stops_cleaned_stop_times_trips_merge_grouped_by_shifted['match'] = stops_cleaned_stop_times_trips_merge_grouped_by_shifted['trip_id'].eq(stops_cleaned_stop_times_trips_merge_grouped_by['trip_id'])
#Drop the rows for which this condition is not satisfied
stops_cleaned_stop_times_trips_merge_grouped_by_shifted.drop(stops_cleaned_stop_times_trips_merge_grouped_by_shifted[stops_cleaned_stop_times_trips_merge_grouped_by_shifted['match'] == False].index, inplace = True)
stops_cleaned_stop_times_trips_merge_grouped_by_shifted = stops_cleaned_stop_times_trips_merge_grouped_by_shifted[['route_id', 'trip_headsign', 'stop_sequence','stop_name']]
stops_cleaned_stop_times_trips_merge_grouped_by_shifted.rename(columns=
  {"stop_sequence": "stop_sequence_1",
  "stop_name": "stop_name_1"}, inplace=True)
#Joins the df with its shifted version sothat each sequence of two stations is represented in the table as a row
df_for_edges = stops_cleaned_stop_times_trips_merge_grouped_by_shifted.join(stops_cleaned_stop_times_trips_merge_grouped_by[['stop_sequence','stop_name']], lsuffix='_caller', rsuffix='_other', how='left')
df_for_edges.rename(columns=
  {"stop_sequence": "stop_sequence_2",
  "stop_name": "stop_name_2"}, inplace=True)

df_for_edges['route_id'] = df_for_edges['route_id'].astype(np.int64)
df_for_edges['stop_sequence_1'] = df_for_edges['stop_sequence_1'].astype(np.int64)
df_for_edges = df_for_edges.drop_duplicates()
df_for_edges

In [ ]:
G = nx.Graph()
G = nx.from_pandas_edgelist(df_for_edges, 'stop_name_1', 'stop_name_2')

* Weights toevoegen

In [ ]:
#Assign node weights to graph
for i in range(len(stops)):
    for n, d in G.nodes(data=True):
        if stops.stop_name[i] == n:
            G.nodes[n]['id']=stops.stop_id[i];
            G.nodes[n]['latitude']=stops.stop_lat[i];
            G.nodes[n]['longitude']=stops.stop_lon[i];
            
G.nodes(data=True)

In [ ]:
#Assign the route_id as a list (because there can be more than one per edge) on each edge
routes_on_edge = {}
for node1,node2, nodeData in G.edges(data = True):
    if (node1, node2) not in routes_on_edge:
            routes_on_edge[(node1, node2)] = set()
    route_id_node1_node2= df_for_edges[(df_for_edges['stop_name_1'] == node1 ) & (df_for_edges['stop_name_2'] == node2 )]['route_id']
    route_id_node2_node1= df_for_edges[(df_for_edges['stop_name_1'] == node2 ) & (df_for_edges['stop_name_2'] == node1 )]['route_id']
    for route_id in pd.concat([route_id_node1_node2, route_id_node2_node1]):
        routes_on_edge[(node1, node2)].add(route_id)
    nodeData['route_id'] = routes_on_edge[(node1, node2)] 

G.edges(data=True)

* Characteristics and global network metrics:




In [ ]:
#aantal nodes
len(G.nodes())

In [ ]:
#degrees van de nodes
G.degree()

In [ ]:
# average degree (indicating the number of directions a commuter can take from that given station)
sum = 0
for n in G.nodes():
    sum = sum + G.degree[n]
print(sum/len(G.nodes()))

In [ ]:
#The existing number of edges
len(G.edges())

In [ ]:
#gamma_index
gamma_index = len(G.edges()) / (3*(len(G.nodes()) - 2))
gamma_index

In [ ]:
#Mean local clustering coefficient
#indicating for a random station the estimation of the division of the number of neighbouring stations of a station that are connected divided by the possible number of connections between the neighbours of that station
print ("mean local clustering:", nx.average_clustering (G))

In [ ]:
#The degree distribution

#Sort degree values
degree_sequence=sorted([d for n,d in G.degree()], reverse=True) 
#Count number of frequences of degree values
degreeCount=collections.Counter(degree_sequence)
#Define variables deg and cnt and probability
deg, cnt = zip(*degreeCount.items())
probability = []
for v in cnt:
    pv= v/len(G.nodes())
    probability.append(pv)

#plot degree probability histogram
fig, ax = plt.subplots(figsize=(5, 5))
plt.bar(deg, probability, width=0.95, color='lightgrey')

#plot degree distribution
plt.plot(deg, probability, color="b")

#graph settings
plt.title("Degree distribution")
plt.ylabel("P (k)")
plt.xlabel("k")
ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

#count
fig, ax = plt.subplots(figsize=(5, 5))
plt.bar(deg, cnt, width=0.95, color='lightgrey')

plt.plot(deg, cnt, color="b")

#graph settings
plt.title("Degree count")
plt.ylabel("Count")
plt.xlabel("k")
ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

plt.show()

In [ ]:
#Detection of the scale-free property (A PTN with the scale-free property consists of a large number of vertices that exhibit low degree and a few number of vertices that exhibit very high degree)

The scale-free property is present in the Belgian railway network because the majority of nodes exhibit a low degree value equal to 2 and almost none of the nodes exhibit a degree value equal to 5 or higher

In [ ]:
nx.draw_networkx(G, with_labels = False, node_size = 5)

In [ ]:
#Average path length (In the L-space it represents the average of all the shortest paths to be transversed between all public transport stop pairs which can be reached by means of one or multiple routes)
#print(nx.average_shortest_path_length(G))

In [ ]:
#Assortativity of the network 
#In an assortative network, high-degree vertices tend to be connected and low-degree vertices tend to stick together as well. 
#In a disassortative network conversely, high-degree and low-degree vertices tend to be connected to each other instead of being connected to vertices of the same kind).

print(nx.degree_pearson_correlation_coefficient(G))

# Onderzoeksvraag 2: Do groups of similar routes in multiple railway systems have a comparable travel time?

# 2.1 Do groups of similar routes in multiple railway systems have a comparable planned travel time?

* Construction of the df route_characteristics

This df contains for each route the aggregated route values

In [ ]:
''' To make a list of all unique route_ids in the df_for_edges dataset'''
list(df_for_edges['route_id'].unique())

In [ ]:
#put all the route_ids in df route_characteristics

''' To create a list of all unique route_ids in the df_for_edges dataset'''
routes_list = list(df_for_edges['route_id'].unique())
routes_list.sort()

''' To create a dictionary where each route of the routes_list represents a key'''
route_characteristics_dic = {'route_id': routes_list}

''' To create a dataframe that contains the values in the route_characteristics_dic as the values in the first column'''
route_characteristics = pd.DataFrame(route_characteristics_dic, columns = ['route_id'])
route_characteristics['route_id'].astype(np.int64)

In [ ]:
#Put the nodes per route in df route_characteristics

''' To construct a new column of the route_characteristics dataset that has for every column value an empty set'''
route_characteristics['nodes'] = [set() for _ in range(len(route_characteristics))]

''' To put the two nodes of an edge in the nodes column of the route_characteristics dataset if the route_id of the corresponding row 
is present in the set of the list values of the corresponding edge that contains all route_ids that contain the edge'''
for node1, node2 in G.edges():
    route_characteristics[['route_id','nodes']].apply(lambda x: x['nodes'].update((node1, node2)) if x['route_id'] in set(G.get_edge_data(node1, node2)['route_id']) else x, axis = 1)
print(route_characteristics)

In [ ]:
for nodes in route_characteristics[route_characteristics['route_id'] == 6]['nodes']:
  print(nodes)

In [ ]:
# The inputs of the get_metric_list function are a dictionary that contains for all node keys the corresponding metric values and a specific route_id of a column of 
# the route_characteristics dataset
# A new empty list metric_list is created for these inputs
# For the nodes that are in the route_characteristics dataset corresponding to this route_id, the corresponding metric values from the dictionary get appended to metric_list

from statistics import mean
def get_metric_list(metric_dic, route_id):
    """this is a sub function that is used in calculate_metric_route_per_node"""
    metric_list = []
    for nodes in route_characteristics[route_characteristics['route_id'] == route_id]['nodes']:
        for node in nodes:
            metric_list.append(metric_dic[node])
    return metric_list

# For the metric, a new column is created in route_characteristics, where every column value consists of an empty list
# The values in the column of the df are determined by giving the metric_dic and the route_id of the row where that value is placed to the get_metric_list function
# The metric_list ultimately contains the metric values for a specific metric for all the stations corresponding to that route

def calculate_metric_route_per_node(metric_dic, metric_name):
    """puts the metrics of each node of a route in a list in the route_characteristics df"""
    route_characteristics[metric_name] = [list for _ in range(len(route_characteristics))]
    route_characteristics[metric_name] = route_characteristics.apply(lambda x: get_metric_list(metric_dic, x['route_id']) , axis = 1)
    
# The function starts from the metric_name, the original_metric_name and the aggregation_method inputs
# A new column named with the metric_name is created: initially it contains all NaN values
# Thereafter the aggregation_method is taken into account. The avg aggregation method returns for all column values the mean of the values in the corresponding row
# of the original_metric_name list. The max aggregation method returns for all column values the max of the values in the corresponding row of the original_metric_name list.

def calculate_metric_aggregation_route(metric_name, original_metric_name, aggregation_method):
    '''calculate the metric aggregation of the routes, only avg and max are supported as string values for aggregation method
    metric_name is the aggregation column name and original_metric_name is the node metric column name'''
    route_characteristics[metric_name] = np.nan
    if aggregation_method == 'avg':
        route_characteristics[metric_name] = route_characteristics.apply(lambda x: mean(x[original_metric_name]) , axis = 1)
    elif aggregation_method == 'max':
        route_characteristics[metric_name] = route_characteristics.apply(lambda x: max(x[original_metric_name]) , axis = 1)

In [ ]:
calculate_metric_route_per_node(dict(G.degree()), 'node_degree')
calculate_metric_aggregation_route('avg_degree', 'node_degree', 'avg')
calculate_metric_aggregation_route('max_degree', 'node_degree', 'max')
calculate_metric_route_per_node(dict(nx.clustering(G)), 'node_clustering')
calculate_metric_aggregation_route('avg_clustering', 'node_clustering', 'avg')
calculate_metric_aggregation_route('max_clustering', 'node_clustering', 'max')
calculate_metric_route_per_node(dict(nx.closeness_centrality(G)), 'node_closeness_centrality')
calculate_metric_aggregation_route('avg_closeness_centrality', 'node_closeness_centrality', 'avg')
calculate_metric_aggregation_route('max_closeness_centrality', 'node_closeness_centrality', 'max')
calculate_metric_route_per_node(dict(nx.betweenness_centrality(G)), 'node_betweenness_centrality')
calculate_metric_aggregation_route('avg_betweenness_centrality', 'node_betweenness_centrality', 'avg')
calculate_metric_aggregation_route('max_betweenness_centrality', 'node_betweenness_centrality', 'max')

In [ ]:
route_characteristics.head(2)

In [ ]:
print(route_characteristics['nodes'][0])

In [ ]:
''' To add the number of stations present in the stop sequence of a route_id'''
number_stations = []
for number_stops in route_characteristics['nodes']:
  number_stations.append(len(number_stops))

route_characteristics['len_stop_sequence'] = number_stations
route_characteristics.head(2)

In [ ]:
stops_cleaned[stops_cleaned['stop_name'] == 'Namur']

In [ ]:
stops_cleaned[stops_cleaned['stop_name'] == 'Bruxelles-Central']

In [ ]:
''' Example: to calculate the distance between Namur and Bruxelles-Central'''
import math

# The radius of the earth
R = 6373.0

# To calculate the coordinates
lat_Namur = math.radians(50.46879)
lon_Namur = math.radians(4.86222)
lat_Brussels = math.radians(50.84565)
lon_Brussels = math.radians(4.356804)

# To calculate the change in coordinates
dlon = lon_Brussels - lon_Namur
dlat = lat_Brussels - lat_Namur

# To use the Haversine formula to get the distance in kilometers between Brussels and Namur
a = math.sin(dlat / 2)**2 + math.cos(lat_Namur) * math.cos(lat_Brussels) * math.sin(dlon / 2)**2
c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

distance = R * c

print(distance)

In [ ]:
''' To add the absolute distances between the starting and the ending station of all routes'''

# Remember that geographical locations that are located further away to the right of the Prime Meridian Greenwich instead of another location that is closer
# (but still to the right) of this Meridian Greenwich, have a greater longitute value.
# Remember that geographical locations that are higher above the Equator than other geographical locations (that are also above the Equator), have a 
# higher latitude value

absolute_distance = []
for route in route_characteristics['route_id']:
  subdf_stop_sequence = df_for_edges[df_for_edges['route_id'] == route]
  number_of_rows = len(subdf_stop_sequence.index)

  # To get the starting station and the ending_station of the stop sequence of a route
  starting_station = subdf_stop_sequence['stop_name_1'].iloc[0]
  ending_station = subdf_stop_sequence['stop_name_2'].iloc[number_of_rows - 1]

  # The radius of the earth
  R = 6373.0

  # To convert the coordinates in radians 
  lat_starting_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == starting_station]['stop_lat'].iloc[0])
  lon_starting_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == starting_station]['stop_lon'].iloc[0])
  lat_ending_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == ending_station]['stop_lat'].iloc[0])
  lon_ending_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == ending_station]['stop_lon'].iloc[0])

  # To calculate the change in coordinates
  dlon = lon_ending_station - lon_starting_station
  dlat = lat_ending_station - lat_starting_station

  # To use the Haversine formula to get the distance in kilometers between the starting_station and the ending_station
  a = math.sin(dlat / 2)**2 + math.cos(lat_starting_station) * math.cos(lat_ending_station) * math.sin(dlon / 2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

  # To calculate the distance
  distance = R * c

  # To add the distance in the absolute_distance list
  absolute_distance.append(distance)

# To add the absolute_distance list as a column to the route_characteristics dataset
route_characteristics['absolute_distance'] = absolute_distance
route_characteristics.head(2)

In [ ]:
''' To calculate the relative distance of the stop sequence of the route by only taking the starting and the ending station into account'''
# The relative distance of a route is the distance of the route divided by the distance of the route with the longest distance

relative_distance = []

# To calculate the route which stop sequence has the greatest absolute distance
max_distance = route_characteristics['absolute_distance'].max()

# To calculate the relative distance of each route
for abs_distance in route_characteristics['absolute_distance']:
  rel_distance = abs_distance / max_distance
  relative_distance.append(rel_distance)

# To add the relative_distance list as a column to the route_characteristics dataset
route_characteristics['relative_distance'] = relative_distance
route_characteristics.head(2)

In [ ]:
route_characteristics.dtypes

* Clustering of the routes

In [ ]:
#Construction of the numpy array from the route_characteristics dataset
arr_route_characteristics = route_characteristics.loc[:, ['avg_degree','max_degree','avg_clustering','max_clustering','avg_closeness_centrality','max_closeness_centrality','avg_betweenness_centrality','max_betweenness_centrality', 'len_stop_sequence', 'absolute_distance', 'relative_distance']].values
arr_route_characteristics

In [ ]:
#Normalization of the arr_route_characteristics
scaler = StandardScaler()
norm_arr_route_characteristics = scaler.fit_transform(arr_route_characteristics)
norm_arr_route_characteristics

In [ ]:
#Find the optimal number of clusters by means of the elbow method
wcss=[]

#a loop to fit the k-means algorithm (of the sklearn library) to the data
#The withing cluster sum of squares is calculated and stored in the wcss list
#init parameter indicates the random initialization method
#the max_iter parameter indicates the maximum number of iterations there can be to find the clusters when running the k-means algorithm
#300 is the default value for this
#The n_init parameter indicates the number of times the k-means algorithm will run with a different initial centroid
#The next line indicates that the k-means algorithms fits to the route_characteristics dataset
#The k-means inertia_ attribute indicates the sum of squared distances of samples to their closest cluster center
for i in range(1,11): 
     kmeans = KMeans(n_clusters=i, init ='k-means++', max_iter=300,  n_init=10,random_state=0 )
     kmeans.fit(norm_arr_route_characteristics)
     wcss.append(kmeans.inertia_)

In [ ]:
#Calculate the optimal number of clusters
kl = KneeLocator(range(1, 11), wcss, curve="convex", direction="decreasing")
kl.elbow

In [ ]:
#Plot the elbow graph
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method Graph')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

kl.plot_knee()

plt.show()

In [ ]:
#Applying k-means algorithm to the route_characteristics dataset.
kmeans = KMeans(n_clusters=kl.elbow, init ='k-means++', max_iter=300, n_init=10,random_state=0 )

#By means of the fit predict method for each observation the cluster it belongs to is returned. 
y_kmeans = kmeans.fit_predict(norm_arr_route_characteristics)
y_kmeans

In [ ]:
#Creation of a dictionary to know which route belongs to which cluster
#Put the routeID column as the index of the dictionary
cl_prediction = pd.Series(index=route_characteristics['route_id']).to_dict()

#select the first key
#list(cl_prediction.keys())[0]

#Put the array values as the values of the dictionary
for i in range(len(y_kmeans)):
  k = list(cl_prediction.keys())[i]
  cl_prediction[k] = y_kmeans[i]

cl_prediction

In [ ]:
#Centers of the clusters
kmeans.cluster_centers_

In [ ]:
#Principal component analysis (PCA) to reduce the number of dimensions
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=33)),
    ])

clusterer = Pipeline(
    [(
        "kmeans",
        KMeans(
            n_clusters= kl.elbow,
            init="k-means++",
            n_init=50,
            max_iter=500,
            random_state=33,
            ),
        ),
        ])

pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
)

pipe.fit(norm_arr_route_characteristics)
preprocessed_data = pipe["preprocessor"].transform(norm_arr_route_characteristics)
predicted_labels = pipe["clusterer"]["kmeans"].labels_

In [ ]:
#Voor centroid values weer te geven
preprocessor2 = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=33)),
    ])

clusterer2 = Pipeline(
    [(
        "kmeans",
        KMeans(
            n_clusters= kl.elbow,
            init="k-means++",
            n_init=50,
            max_iter=500,
            random_state=33,
            ),
        ),
        ])

pipe2 = Pipeline(
    [
        ("preprocessor2", preprocessor2),
        ("clusterer2", clusterer2)
    ]
)

pipe2.fit(kmeans.cluster_centers_)
preprocessed_data_2 = pipe2["preprocessor2"].transform(kmeans.cluster_centers_)
predicted_labels_2 = pipe2["clusterer2"]["kmeans"].labels_

In [ ]:
#Plot the centroids
cl_routes_2 = pd.DataFrame(
    pipe2["preprocessor2"].transform(kmeans.cluster_centers_),
    columns=["Principal component 1", "Principal component 2"],
)

#cl_routes_2["predicted_cluster"] = pipe2["clusterer2"]["kmeans"].labels_

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=300,
    data=cl_routes_2,
    color=".2", 
)

scat.set_title("Clusters of the Belgian railway centroids")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

In [ ]:
#Plot the clusters
cl_routes = pd.DataFrame(
    pipe["preprocessor"].transform(norm_arr_route_characteristics),
    columns=["Principal component 1", "Principal component 2"],
)

cl_routes["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_

cl_routes_2 = pd.DataFrame(
    pipe2["preprocessor2"].transform(kmeans.cluster_centers_),
    columns=["Principal component 1", "Principal component 2"],
)

cl_routes_2["predicted_cluster"] = pipe2["clusterer2"]["kmeans"].labels_

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=50,
    data=cl_routes,
    hue="predicted_cluster",
    palette="Set2",
)

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=300,
    data=cl_routes_2,
    color=".2", 
)

scat.set_title("Clusters of the Belgian railway routes")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

In [ ]:
#Plot the clusters
cl_routes = pd.DataFrame(
    pipe["preprocessor"].transform(norm_arr_route_characteristics),
    columns=["Principal component 1", "Principal component 2"],
)

cl_routes["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=50,
    data=cl_routes,
    hue="predicted_cluster",
    palette="Set2",
)

scat.set_title("Clusters of the Belgian railway routes")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

# Testen van de osmnx package

In [ ]:
ox.plot_graph(ox.graph_from_place('Namur, Belgium'))

In [ ]:
B = ox.graph_from_place('Belgium',
                        retain_all=False, truncate_by_edge=True, simplify=True,
                        custom_filter='["railway"~"rail"]')

fig, ax = ox.plot_graph(B, node_size=0, edge_color='w', edge_linewidth=0.2)